## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-24-16-46-31 +0000,nyt,"In Michigan’s Upper Peninsula, Locals Fight fo...",https://www.nytimes.com/2025/11/24/science/ast...
1,2025-11-24-16-42-01 +0000,nyt,Russia’s Invasion of Ukraine Prompts Debate Ov...,https://www.nytimes.com/2025/11/24/business/ec...
2,2025-11-24-16-40-26 +0000,nyt,"Dharmendra, Bollywood Leading Man, Dies at 89",https://www.nytimes.com/2025/11/24/world/asia/...
3,2025-11-24-16-37-27 +0000,nyt,Putin’s Win-Win: Take a Russia-Friendly Peace ...,https://www.nytimes.com/2025/11/24/world/europ...
4,2025-11-24-16-36-44 +0000,nyt,"X Displays Users’ Locations, Fueling Scrutiny ...",https://www.nytimes.com/2025/11/24/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,ukraine,23
30,trump,21
21,peace,15
293,plan,14
7,russia,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
141,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,101
140,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,85
218,2025-11-23-17-38-05 +0000,nypost,Zelensky thanks Trump for support after critic...,https://nypost.com/2025/11/23/world-news/zelen...,85
48,2025-11-24-14-24-48 +0000,nyt,U.S. and Ukraine Hail Progress on Peace Plan t...,https://www.nytimes.com/2025/11/24/world/europ...,82
177,2025-11-23-23-12-21 +0000,nypost,Leaked EU peace deal calls for Ukraine to keep...,https://nypost.com/2025/11/23/world-news/leake...,80


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
141,101,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...
207,34,2025-11-23-19-31-39 +0000,bbc,Israel kills top Hezbollah official in first a...,https://www.bbc.com/news/articles/cn81j54xjx1o...
182,32,2025-11-23-22-31-51 +0000,nypost,Canada’s prime minister says trade talks with ...,https://nypost.com/2025/11/23/business/canadas...
166,31,2025-11-24-01-11-46 +0000,nyt,"Lee Tamahori, Director of Film Voted New Zeala...",https://www.nytimes.com/2025/11/23/obituaries/...
79,29,2025-11-24-11-58-48 +0000,nypost,Outgoing Rep. Marjorie Taylor Greene shoots do...,https://nypost.com/2025/11/24/us-news/outgoing...
83,28,2025-11-24-11-39-26 +0000,nypost,Erika Kirk reveals she prayed she was pregnant...,https://nypost.com/2025/11/24/us-news/erika-ki...
101,28,2025-11-24-11-00-00 +0000,wapo,More Americans are getting their power shut of...,https://www.washingtonpost.com/business/2025/1...
28,25,2025-11-24-15-40-02 +0000,nypost,‘Adult accomplice’ busted with ‘Slender Man’ s...,https://nypost.com/2025/11/24/us-news/woman-42...
211,23,2025-11-23-18-53-10 +0000,cbc,Bolsonaro claims 'hallucinations' led the ex-B...,https://www.cbc.ca/news/world/bolsonaro-brazil...
21,22,2025-11-24-15-53-49 +0000,nypost,Vanity Fair to publish ‘abstract nude portrait...,https://nypost.com/2025/11/24/media/vanity-fai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
